<a href="https://colab.research.google.com/github/nazuna2371/ai_synthesized_voice/blob/main/tacotron2_output_side.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **<font color= "blue">ドライブマウント・認証</font>**

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **<font color= "blue">ライブラリインポート</font>**

%%bash
pip install numpy scipy librosa unidecode inflect librosa
apt-get update
apt-get install -y libsndfile1

In [ ]:
import os
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/sound_source_separation/tacotron2")
from quote_list import quote_list_table
import torch
from IPython.display import Audio
import torchaudio

In [ ]:
%cd /content
!ln -s "/content/drive/MyDrive/Colab Notebooks/model_storage/self_create/tacotron2/outdir_2" .
!ls -lah .

/content
ln: failed to create symbolic link './outdir_2': File exists
total 24K
drwxr-xr-x 1 root root 4.0K Jul 17 12:45 .
drwxr-xr-x 1 root root 4.0K Jul 17 12:35 ..
drwxr-xr-x 4 root root 4.0K Jul 13 13:32 .config
drwx------ 5 root root 4.0K Jul 17 12:45 drive
lrwxrwxrwx 1 root root   83 Jul 17 12:45 outdir_2 -> '/content/drive/MyDrive/Colab Notebooks/model_storage/self_create/tacotron2/outdir_2'
drwxr-xr-x 1 root root 4.0K Jul 13 13:33 sample_data


In [ ]:
#@title **<font color= "blue">nvidia_waveglow 読み込み</font>**

waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow.cuda().eval().half()

In [ ]:
#@title **<font color= "blue">pyopenjtalkで作成したテキスト読み込み</font>**

target_directory = "outdir_2/"
cp_list = os.listdir(target_directory)
text_list = quote_list_table().sentence_array

In [ ]:
#@title **<font color= "blue">選択したテキストから推論実行</font>**

target_cp = cp_list[8] # 音声学習済モデル
text = text_list[4] # wakariyasUkutsUtaerutamenikajoogakioriyoosurunowa,iclpaNtekinazaclshiyapaNfurecltonadooheNshuusurusainimotsUkawarerugihoodesU.

check_point = target_directory + target_cp

tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2.load_state_dict(torch.load(check_point)['state_dict'])
tacotron2.cuda().eval().half()

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [ ]:
#@title **<font color= "blue">生成された合成ボイス確認</font>**

Audio(audio_numpy, rate=rate)

In [ ]:
#@title **<font color= "blue">合成ボイス保存</font>**

audio_tensor = torch.from_numpy(audio_numpy).float().unsqueeze(0)
torchaudio.save(filepath='/content/outdir_2/tmp/audio_1.wav', src=audio_tensor, sample_rate=rate)